In [1]:
import pandas as pd
import numpy as np
import json
import requests
from bs4 import BeautifulSoup

In [2]:
data = ("C:\\Users\\yahav\\MyNotebooks\\destinations_LP_crawler_Ex5.csv")

In [3]:
df = pd.read_csv(data,encoding='ISO-8859-1')
df = df.replace(to_replace='Krakï¿', value='krakow', regex=True) ## replace the invalid city
df

,city,city_LP
0,Antalya,mediterranean-coast/antalya
1,Athens,Athens
2,Ayia-napa,the-republic-of-cyprus/Agia-napa
3,Bangkok,Bangkok
4,Barcelona,Barcelona
...,...,...
71,Venice,Venice
72,Vienna,Vienna
73,Warsaw,Warsaw
74,Zakynthos,Zakynthos


## Finding the country's name

In [4]:
lst = list()
address = list()
for i in df['city']:
    address.append(i)

In [5]:
api_key = 'Please enter your API key' ##My API key

In [6]:
def get_location(address): ## function to the country's name
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s' % (address,api_key)
    try:
        response = requests.post(url)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response_a = response.json()
                sentence_len = len(response_a["results"][0]['address_components'])-1
                c_name = response_a["results"][0]['address_components'][sentence_len]['long_name']
                return c_name
                try:
                    sentence_split = int(c_name.split()[0])
                    c_name = response_a["results"][0]['address_components'][sentence_len-1]['long_name']
                    print(c_name)
                except:
                    print(c_name)
            except:
                response_a = requests.get(url)
                response_a = response.json()
                sentence_len = len(response_json["results"][0]['address_components'])-2
                c_name = response_a["results"][0]['address_components'][sentence_len]['long_name']
                print(c_name)
    except:
        print("Something went wrong with requests.get")

In [7]:
countrylst = list()
for i in address:
    countrylst.append(get_location(i))
for i in range(len(countrylst)): #Changing the incorrect names
    if countrylst[i] == 'United States':
        countrylst[i] = 'usa'
    elif countrylst[i] == '30':
        countrylst[i] = 'poland'
    elif countrylst[i] == '02000':
        countrylst[i] = 'ukraine'
    elif countrylst[i] == '65000':
        countrylst[i] = 'ukraine' 
    elif countrylst[i] == '846 00':
        countrylst[i] = 'Greece'
    elif countrylst[i] == '03':
        countrylst[i] = 'poland'
    elif countrylst[i] == '83000':
        countrylst[i] = 'Thailand'
        
#print(countrylst)

# Description

In [8]:
new_data = pd.DataFrame() ## new DF
new_data['City'] = df['city']
new_data['Country'] = countrylst
new_data

,City,Country
0,Antalya,Turkey
1,Athens,Greece
2,Ayia-napa,Cyprus
3,Bangkok,Thailand
4,Barcelona,Spain
...,...,...
71,Venice,Italy
72,Vienna,Austria
73,Warsaw,poland
74,Zakynthos,Greece


In [9]:
link = list()
for country,LP in zip(new_data['Country'], df['city_LP']):
    web_link = "https://www.lonelyplanet.com/" + country + "/" + LP
    link.append(web_link)
#print(link)

In [10]:
def get_description(link):
    discription = list()
    try:
        response = requests.get(link)
        if not response.status_code == 200:
            discription.append("HTTP error",response.status_code)
        result_page = BeautifulSoup(response.content,'html.parser')  
        for sentence in result_page.find_all('div',class_= "readMore_content___5EuR"):
            discription.append(sentence.get_text().replace('/n',' '))
            return " ".join(discription)
    except:
        return "Something went wrong"


In [11]:
discriptionlst = list()
for i in link:
    discriptionlst.append(get_description(i))
#discriptionlst


In [12]:
new_data["Description"] = discriptionlst
new_data

,City,Country,Description
0,Antalya,Turkey,Once seen simply as the gateway to the Turkish...
1,Athens,Greece,"With equal measures of grunge and grace, Athen..."
2,Ayia-napa,Cyprus,Endless blue-sky days and a coastline riddled ...
3,Bangkok,Thailand,"Same same, but different. This Thailish T-shir..."
4,Barcelona,Spain,Barcelona is an enchanting seaside city with b...
...,...,...,...
71,Venice,Italy,Imagine the audacity of building a city of mar...
72,Vienna,Austria,Baroque streetscapes and imperial palaces set ...
73,Warsaw,poland,"A phoenix arisen from the ashes, Poland's capi..."
74,Zakynthos,Greece,"Zakynthos, also known by its Italian name Zant..."


In [13]:
new_data.to_csv("C:\\Users\\yahav\\MyNotebooks\\LP_destinations.csv",index=False)
